In [ ]:
import numpy as np
import pandas as pd, os
from sklearn.feature_extraction import DictVectorizer

def one_hot_dataframe(data, cols, replace=False):
    vec = DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    len1 = len(vec.get_feature_names())
    #print(len1)
    vecData.columns = vec.get_feature_names()
    del vecData[vec.get_feature_names()[len1-1]]
    #print(vecData)
    vecData.index = data.index
    if replace is True:
        #print(data)
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
        #print(data)
    return (data, vecData, vec)

In [ ]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'nation': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}


df = pd.DataFrame(data)

df2, _, _ = one_hot_dataframe(df, ['state','nation'], replace=True)
print (df2)

In [ ]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'nation': ['India', 'India', 'IndiaUSA', 'USA', 'USA'],
        'year': [2000, 2001, 2002, 2001, 2002],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}

In [ ]:
import csv
def readTrainingData():
    data = {}
    colNames = []
    categories = []
    continuous = []
    target = []
    target1 = []
    with open('../input/train.csv') as csvfile:
        trainReader = csv.reader(csvfile, delimiter=',')
        count = 0
        for row in trainReader:
            if count == 0:
                colNames = row
                count+=1
            else:
                #print(row[0])
                key = int(row[0])
                row.pop(0)
                categories.append(row[0:116])
                continuous1 = row[116:130]
                target1.extend(row[130:131])
                idxToBeDeleted = len(row) - 1
                row.pop(idxToBeDeleted)
                #print(row)
                continuous1 = [ float(x) for x in continuous1 ]
                continuous.append(continuous1)
                data[key] = row
        #print(categories)
        #print(continuous)
        #print(target)
        for item in target1:
            target.append(float(item))
    return data, categories,continuous,target



#df = pd.DataFrame(data)

#df2, _, _ = one_hot_dataframe(df, ['state','nation'], replace=True)
#print (df2)

In [ ]:
data, categories,continuous,target = readTrainingData()
print (categories[0])

In [ ]:
print (categories[0])

In [ ]:
def categoryEncoder(categories,continuous): 
    uniqCatAndCounts = {}
    numRows = len(categories)
    numCols = len(categories[0])
    encodedCategories = [[0 for x in range(numCols)] for y in range(numRows)] 
    #print("numRows:"+str(numRows))
    #print("numCols:"+str(numCols))
    for i in range(numRows):
        for j in range(numCols):
            if categories[i][j] in uniqCatAndCounts:
                uniqCatAndCounts[categories[i][j]] += 1
            else:
                uniqCatAndCounts[categories[i][j]] = 1
    label = 1
    for cat in uniqCatAndCounts:
        uniqCatAndCounts[cat] = label
        label += 1
    for i in range(numRows):
        for j in range(numCols):
            if categories[i][j] in uniqCatAndCounts:
                encodedCategories[i][j] = float(uniqCatAndCounts[categories[i][j]])
        encodedCategories[i].extend(continuous[i])
    return encodedCategories,uniqCatAndCounts

encodedCategories,uniqCatAndCounts = categoryEncoder(categories,continuous)
print (encodedCategories[0])

In [ ]:
import csv
def readTestData():
    data = {}
    colNames = []
    categories = []
    continuous = []
    count = 0
    with open('../input/test.csv') as csvfile:
        testReader = csv.reader(csvfile, delimiter=',')
        for row in testReader:
            if count == 0:
                colNames = row
                count+=1
            else:
                key = int(row[0])
                row.pop(0)
                categories.append(row[0:116])
                continuous1 = row[116:130]
                continuous1 = [ float(x) for x in continuous1 ]
                continuous.append(continuous1)
                data[key] = row
    return data, categories, continuous

dataTest, categoriesTest, continuousTest = readTestData()
print(categoriesTest[0])
#print(continuousTest[0])

In [ ]:
def testCatEncoder(categoriesTest, uniqCatAndCounts, continuousTest):
    #encodedCategoriesTest = categoriesTest
    numRows = len(categoriesTest)
    numCols = len(categoriesTest[0])
    label = len(uniqCatAndCounts) + 1
    encodedCategoriesTest = [[0 for x in range(numCols)] for y in range(numRows)] 
    for i in range(numRows):
        for j in range(numCols):
            if categoriesTest[i][j] in uniqCatAndCounts:
                encodedCategoriesTest[i][j] = float(uniqCatAndCounts[categoriesTest[i][j]])
            else:
                #Should update uniqCatAndCounts with new key and new value
                uniqCatAndCounts[categoriesTest[i][j]] = label
                encodedCategoriesTest[i][j] = label
                label += 1
        encodedCategoriesTest[i].extend(continuousTest[i])
    return uniqCatAndCounts, encodedCategoriesTest

uniqCatAndCounts, encodedCategoriesTest = testCatEncoder(categoriesTest, uniqCatAndCounts, continuousTest)
print(encodedCategoriesTest[0])

In [ ]:
import numpy as np
from sklearn import linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(encodedCategories, target)

# The coefficients
#print('Coefficients: \n', regr.coef_)

# The mean squared error
#print("Mean squared error: %.2f"
      #% np.mean((regr.predict(diabetes_X_test) - diabetes_y_test) ** 2))
targetTest = regr.predict(encodedCategoriesTest)
print(len(targetTest))

In [ ]:
print(targetTest)

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

# Split the data into training/testing sets
encodedCategories_X_train = encodedCategories[:-20]
encodedCategories_X_test = encodedCategories[-20:]

# Split the targets into training/testing sets
target_y_train = target[:-20]
target_y_test = target[-20:]

#print(target_y_test)

# Create linear regression object
regr1 = linear_model.LinearRegression()

# Train the model using the training sets
regr1.fit(encodedCategories_X_train, target_y_train)

# The mean squared error
#print("Mean squared error: %.2f"
      #% np.mean((regr.predict(encodedCategories_X_test) - target_y_test) ** 2))
mean_squared_error(regr.predict(encodedCategories_X_test), target_y_test)
#print(regr1.predict(encodedCategories_X_test))
#print(target_y_test)

In [ ]:
import pandas as pd
from sklearn import preprocessing

le_cat1 = preprocessing.LabelEncoder()

train = pd.read_csv('../input/train.csv')
#train.cat1 = le_cat1.fit_transform(train.cat1)
#print(train)
dummies = pd.get_dummies(train)



In [ ]:
#print(dummies['cat1_A'])


In [ ]:
import pandas as pd
from sklearn import preprocessing

enc = preprocessing.OneHotEncoder()
enc.fit(encodedCategories)
#enc.transform([[0, 1, 3]]).toarray()

In [ ]:
import pandas as pd
from sklearn import preprocessing

enc = preprocessing.OneHotEncoder()
enc.fit(encodedCategories)
print("Hi")
#enc.transform([[0, 1, 3]]).toarray()

In [ ]:
print (len(enc.transform([encodedCategories[0]]).toarray()))